In [1]:
!pip install PyPDF2
!pip install pandas

In [1]:
import re
import PyPDF2 as pdf
import pandas as pd
import numpy as np
import math 

In [24]:
# AAPL IS DONE 
# ADBE IS DONE 
# start from BA

In [40]:
#Howdy Script User, this script follows a natural progression in which you first grab the table of contents, 
#grab data from that table of contents, grab the needed pages from a a page-range list and then analyze each er document
#getting the needed information out of it

## FOR MSFT , make sure to skip document number 8 
# if(index == 8):
#            continue 

#Data will be gathered by SMP 500 selected stock. :^) 

#!Important - make sure you fill up every single data dictionary space!

## just enter in proper ticker and the number of equity research reports
def get_TOC(ticker, num_docs):
    for index in range(1, (num_docs+1)):
        export_path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_data_tc_txt/{0} ({1}).pdf".format(ticker, index)
        path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0} ({1}).pdf".format(ticker,index)
        export_doc = pdf.PdfFileWriter()
        doc = pdf.PdfFileReader(open((path),"rb"))
        for place in range(7):  
            page = doc.getPage(place)
            try:
                test = page.extractText()[1]
                export_doc.addPage(page)
            except:
                pass
        export_doc.write(open(export_path,"wb"))
    print("I'm done!")
get_TOC("NFLX",5)

I'm done!


In [56]:
#///////////////////////////////////// IMPORTANT, these need to be defined here so they can be used in the rest of the script
data = {'Bank':[], 'Date':[], 'Position':[],'Text':[],'Price Target':[]}
pageRange_list = {}
#/////////////////////////////////////


def get_Pages(string):
    index = int(len(string)*(0.7))
    string_list = string[index:].split(' - ')
    if(len(string_list)<2):
        string_list[0] = "DNE"
    else:
        try:
            string_list[0] = int(string_list[0][-3:].strip())
        except:
            string_list[0] = 'DNE'
        arr = string_list[1].split(" ")
        try:
            string_list[1] = int(arr[0].strip())
        except:
            try:
                string_list[1] = string_list[0] + 6
            except: 
                string_list[1] = 'DNE'
    return string_list
    
def get_DateAndBank(string):
    try:
        string_list = string.split("     ")
    except:
        string_list = ["DNE"]
    if(len(string_list) != 2):
        return ['DNE']
    else:
        return string_list


start_string = "Table of Contents"
end_string = "These reports were compiled using a product of Thomson Reuters"    

def TOC_scrape(ticker, num_docs):
    er_report = 1
    for index in range(1, num_docs+1):
        pageRange_list[er_report] = []
        file_path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_data_tc_txt/txt_files/{0} ({1}).txt".format(ticker, index)
        with open(file_path,'r', errors='ignore') as file:
            file_text = file.readlines()
            pages = {}
            page_index = 1
            record_pages = False
            for line in file_text:
                try:
                    test = line.index(start_string)
                    pages[page_index] = []     
                    record_pages = True
                    continue
                except:
                    try:
                        test = line.index(end_string)
                        record_pages = False
                        page_index += 1
                        continue
                    except: 
                        pass 
                if((line!='\n') and record_pages):
                    pages[page_index].append(line.strip())       #puts all relavent TOC pages in a list, list then stored into a
                                                                #dictionary - organized by document number.
                    
            counter = 0
            boole = False
            temp_dict = {}                                       #further sorts list into a dictionary seperating each report
            for key in pages:                                   #into a temporary dictionary in which each report overview is then analyzed
                for key_data in (pages[key]):                   #page numbers put into PageRange_list dict as a tuple, date and bank put into MAIN DATA DICTIONARY
                    try:
                        test = key_data.index("Rpt.")
                        boole = True  
                    except:
                        boole = False
                    if(boole):    
                        if((not temp_dict)):    #if the dictionary holds something
                            counter += 1
                            temp_dict.clear()
                            temp_dict["rpt"+str(counter)] = []
                            temp_dict["rpt"+str(counter)].append(key_data)  
                        else:
                            counter += 1
                            temp_dict["rpt"+str(counter)] = []
                            temp_dict["rpt"+str(counter)].append(key_data)         
                    else:
                        temp_dict["rpt"+str(counter)].append(key_data)
               #now we have a temp dict in which we can do operations on
                for report in temp_dict:
                    report_info = temp_dict[report]
                    length = len(report_info)
                    if(length>3):                #abnormal conditions
                        page_pair = get_Pages(report_info[0])
                        if(page_pair[0] == 'DNE'):
                            try:
                                fixed_page = pageRange_list[er_report][-1][1] + 1
                            except: 
                                fixed_page = 6
                            pageRange_list[er_report].append((fixed_page,fixed_page+4))
                        else:    
                            pageRange_list[er_report].append((page_pair[0], page_pair[1])) #tuple added to pageRange_list
                        
                        date_bank = get_DateAndBank(report_info[2])
                        if(date_bank[0] == 'DNE'):
                            date_bank.append('DNE')
                        data['Bank'].append(date_bank[1])
                        data['Date'].append(date_bank[0])
                        
                             
                    else:                        #normal conditions   
                        page_pair = get_Pages(report_info[0])
                        if(page_pair[0] == 'DNE'):
                            try:
                                fixed_page = pageRange_list[er_report][-1][1] + 1
                            except: 
                                fixed_page = 6
                            pageRange_list[er_report].append((fixed_page,fixed_page+4))
                        else:    
                            pageRange_list[er_report].append((page_pair[0], page_pair[1])) #tuple added to pageRange_list
                        
                        date_bank = get_DateAndBank(report_info[1])
                        if(date_bank[0] == 'DNE'):
                            date_bank.append('DNE')
                        data['Bank'].append(date_bank[1])
                        data['Date'].append(date_bank[0])
            
#                 if(er_report == 6):
#                     print(temp_dict)
                temp_dict.clear()
        
        er_report += 1 
    
TOC_scrape("XOM",5)
print(data)
print(pageRange_list)

{'Bank': ['EVERCORE ISI', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'MORGAN STANLEY', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'EVERCORE ISI', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'MORGAN STANLEY', 'MORGAN STANLEY', 'JPMORGAN', 'EVERCORE ISI', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'JPMORGAN', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'EVERCORE ISI', 'JEFFERIES', 'JPMORGAN', 'EVERCORE ISI', 'JPMORGAN', 'JEFFERIES', 'JPMORGAN', 'JPMORGAN', 'E

In [57]:
## THIS IS FOR PREPROCESSING RAW PDFS INTO pdfs that have specified pages and converts them into text files 
# input the page range dictionary into this script

#3RD STEP
new_addresses = "_data.pdf"

def get_pages(ticker, pageRangeDict): 
    splitter_path = "/Users/Ethan Ohman/Desktop/Equity Research/splitter_new.pdf"
    splitter = pdf.PdfFileReader(open(splitter_path,"rb")).getPage(0) # research paper splitter "/-/-/-/-/-/-/-/-/-/-/"
    for report_index in range(1, len(pageRangeDict)+1):
        path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0} ({1}).pdf".format(ticker,report_index)
        er_paper = pdf.PdfFileReader(open(path,"rb"))
        export_data = pdf.PdfFileWriter()  #now you have a writer for all said data
        for page_pairs in pageRangeDict[report_index]:       #pageRangeDict[report_index] returns a tuple (guaranteed to work)
            page_start = page_pairs[0]
            page_end = page_pairs[1]
            page_range = int(math.floor((page_end-page_start)))
            if(page_range < 1):
                page_range = 1
            for index in range(page_start-1,(page_start-1)+page_range):
                page = er_paper.getPage(index)
                export_data.addPage(page)
            export_data.addPage(splitter)
        export_path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_data/{0} ({1})_data.pdf".format(ticker,report_index)  
        print(report_index)
        export_data.write(open(export_path, "wb"))
    print('CONVERSION IS DONE!')
        
        
        ### parser should be done -- test later :) 

get_pages('XOM', pageRange_list)

1
2
3
4
5
CONVERSION IS DONE!


In [205]:
#//////////////////////////////////////////////EVERCORE
def evercoreParser(bankName,report):
    if(bankName!='EVERCORE ISI'):
        return ('DNE','DNE','DNE')
    #ADD INITIALIZERS
    position = 'DNE'
    price_target = 'DNE'
    #Position
    #Price Target
    for index in range(10):
        try:
            report[index].index('TARGET PRICE')
            info = report[index].split('|')
            position = info[0].strip()
            pt = info[1].split('$')
            pt = pt[1][:6]
            try:
                price_target = float(pt.strip())
            except:
                price_target = pt[1].strip()
            break
        except:
            pass       
    #Text
    text = ''
    for line in report:
        text += " " + line.strip()
    data_tuple = (position, price_target, text)
    return data_tuple

#//////////////////////////////////////////////OPPENHIEMER
def oppenhiemerParser(bankName,report):
    if(bankName!='OPPENHEIMER AND CO'):
        return ('DNE','DNE','DNE')
    #ADD INITIALIZERS
    position = 'DNE'
    price_target = 'DNE'
    #Position
    for index in range(10):
        try:
            report[index].index('Stock Rating:')
            position = report[index+2].strip()
    #Price Target
            pt = report[index+3].split('12-18 mo. Price Target') 
            price_target = pt[1].strip()
            try:
                price_target = float(price_target)
                break
            except:
                break
        except:
            pass
    #Text
    text = ''
    for line in report:
        text += " " + line.strip()
        
    data_tuple = (position, price_target, text)
    return data_tuple
        

#//////////////////////////////////////////////JPMORGAN
def jpmoragnParser(bankName,report):
    if(bankName!='JPMORGAN'):
        return ('DNE','DNE','DNE')
    #ADD INITIALIZERS
    position = 'DNE'
    price_target = 'DNE'
    #Position
    for index in range(20):
        try:
            report[index].index('Price (')
            position = report[index-2].strip()
        except:
            pass
        #Price Target
        try:
            report[index].index('Price Target')
            temp_list = report[index].split('$')
            price_target = float(temp_list[1].strip())
            break
        except:
            pass
    #Text
    text = ''
    for lines in report:
        text += " " + lines.strip()

    data_tuple = (position, price_target, text)
    return data_tuple

#//////////////////////////////////////////////JEFFERIES
def jefferiesParser(bankName,report):      #for this one, there are two types of documents, the old version, and the new version
    if(bankName!='JEFFERIES'):
        return ('DNE','DNE','DNE')
    #ADD INITIALIZERS
    position = 'DNE'
    price_target = 'DNE'
    #//// NEW TYPE
    #Position
    oldreport = True
    for index in range(20):
        try:
            #Position
            report[index].index('RATING')
            temp_list = report[index].split('RATING')
            position = temp_list[1].strip()
            continue
        except:
            try:
                #Price Target
                report[index].index('PRICE TARGET (PT)')
                try:
                    temp_list = report[index].split('$')
                    price_target = temp_list[1].strip()
                    try:
                        price_target = float(price_target)
                    except:
                        pass
                    oldreport = False
                    break
                except:
                    temp_list = report[index-1].split('$')
                    price_target = temp_list[1].strip()
                    try:
                        price_target = float(price_target)
                    except:
                        pass
                    oldreport = False
                    break     
            except:
                pass
    if(oldreport):
        for index in range(10):
            try:
                #Position
                report[index].index(')')
                l = report[index].split(')')
                position = l[1].strip()
                continue
            except:
                try:
                    #Price Target
                    report[index].index('Price target')
                    p = report[index].split('$')
                    price_target = p[1].strip()
                    price_target = float(price_target)
                    break
                except:
                    pass   
    #Text 
    text = ''
    for lines in report:
        text += " " + lines.strip()   
    data_tuple = (position, price_target, text)
    return data_tuple

#//////////////////////////////////////////////MORGANSTANLEY
def morganStanParser(bankName,report):
    if(bankName!='MORGAN STANLEY'):
        return ('DNE','DNE','DNE')
    #ADD INITIALIZERS
    position = 'DNE'
    price_target = 'DNE'
    recent_report = False
    old_report = False
    try:
        report[0].index('|')
        old_report = True
    except:
        recent_report = True
    
    if(recent_report):
        for index in range(20):
            try:
                report[index].index('Stock Rating')
                try:
                    report[index+1].index('$')
                    temp_list = report[index+1].split('$')
                    position = temp_list[0][:15].strip()
                    price_target = temp_list[1][:7].strip()
                    try:
                        price_target = float(price_target)
                    except:
                        pass
                    break
                except:
                    temp_list = report[index+2].split('$')
                    position = temp_list[0][:15].strip()
                    price_target = temp_list[1][:7].strip()
                    try:
                        price_target = float(price_target)
                    except:
                        pass
                    break
            except:
                pass
    if(old_report):
        for index in range(15,30):
            try:
                report[index].index('Stock Rating')
                temp_list = report[index].split('Stock Rating')
                position = temp_list[1].strip()
                try:
                    report[index+2].index('$')
                    temp_list = report[index+2].split('$')
                    pt = temp_list[1].strip()
                    price_target = float(pt)
                    break
                except:
                    temp_list = report[index+4].split('$')
                    pt = temp_list[1].strip()
                    price_target = float(pt)
                    break
            except:
                pass
    
    #Text 
    text = ''
    for lines in report:
        text += " " + lines.strip()  
    data_tuple = (position, price_target, text)
    return data_tuple

    
    
    
def switch(argument,report):
    cases = {
        'JPMORGAN': jpmoragnParser(argument, report),
        'EVERCORE ISI': evercoreParser(argument, report),
        'MORGAN STANLEY': morganStanParser(argument, report),
        'OPPENHEIMER AND CO': oppenhiemerParser(argument, report),
        'JEFFERIES': jefferiesParser(argument, report)
    }
    data_tuple = cases.get(argument, ('DNE','DNE','DNE'))
    return data_tuple
    
def equityResearchParser(ticker, num_docs):
    counter = 0 #counter to keep track
    for section in range(1,num_docs+1):
        path = "/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_data_txt/{0} ({1})_data.txt".format(ticker,section)
        er_report = []
        with open(path,'r',errors='ignore') as er_collection:
            full_document = er_collection.readlines()
            for lines in full_document:
                if(lines != '\n'):
                    try:
                        lines.index("/-/-/-/-/-/-/-/-/-/-/NEWPAGE")
                        bank = data['Bank'][counter]
                        data_tuple = switch(bank, er_report)
                        # Position
                        data['Position'].append(data_tuple[0])
                        #Text
                        data['Text'].append(data_tuple[2])
                        # Price Target
                        data['Price Target'].append(data_tuple[1])
                        counter += 1
                        er_report.clear()
                        continue
                        
                    except:
                        lines.strip()
                        er_report.append(lines[:-1])
        
    
equityResearchParser('XOM',5)
print(data['Position'])
count = 0
for mark in data['Position']:
    if(mark == 'DNE'):
        count += 1
print(count)
print(len(data['Bank']))


['In Line', 'In Line', 'HOLD', 'DNE', 'Neutral', 'Neutral', 'DNE', 'Neutral', 'In Line', 'Equal-weight', 'In Line', 'Neutral', 'HOLD', 'In Line', 'DNE', 'Neutral', 'Neutral', 'DNE', 'In Line', 'In Line', 'HOLD', 'DNE', 'DNE', 'Neutral', 'Neutral', 'DNE', 'DNE', 'Neutral', 'In Line', 'DNE', 'DNE', 'In Line', 'HOLD', 'DNE', 'DNE', 'HOLD', 'DNE', 'In Line', 'HOLD', 'DNE', 'In Line', 'DNE', 'Bloomberg NYSE: XOM', 'DNE', 'DNE', 'DNE', 'IN LINE', 'HOLD', 'DNE', '', 'DNE', 'DNE', 'IN LINE', 'DNE', 'HOLD', 'IN LINE', 'DNE', '', 'DNE', 'DNE', 'DNE', 'DNE', 'IN LINE', 'DNE', '', 'DNE', 'IN LINE', 'HOLD', 'DNE', 'IN LINE', 'DNE', '', 'DNE', 'DNE', 'IN LINE', 'HOLD', 'DNE', 'IN LINE', 'DNE', '', 'DNE', 'Underweight', 'DNE', 'IN LINE', 'DNE', '', 'IN LINE', 'HOLD', 'DNE', 'IN LINE', 'DNE', 'Underweight', 'DNE', 'DNE', 'DNE', 'HOLD', 'IN LINE', 'Underweight', 'IN LINE', 'DNE', 'DNE', 'DNE', 'DNE', 'BUY', 'Underweight', 'DNE', 'DNE', 'DNE', 'BUY', 'HOLD', 'BUY', 'DNE', 'DNE', '', 'Underweight', 'DNE'

In [56]:
def email_return(unfil_string):
    email_list = []
    instances = [loc.start() for loc in re.finditer('@', unfil_string)]
    for index in range(instances[0],-1,-1):
        if(unfil_string[index]==' '):
            mold_string = unfil_string[index:].strip()
            break
        if(index == 0):
            mold_string = unfil_string.strip()
    for loc in instances:
        end = mold_string.find(' ')
        if(end == -1):
            email_list.append(mold_string)
            break
        email_list.append(mold_string[:end])
        mold_string = mold_string[end:].strip()
    return email_list   
        
# WORKS!!
list_of_returns = []        
list_of_returns.append(email_return('Andrew.Uerkwitz@opco.com             Martin.Yang@opco.com'))
list_of_returns.append(email_return('Paul.Dean@opco.com'))
list_of_returns.append(email_return('                                                                                      James.Faucette@morganstanley.com            +1 212 296-5771'))
list_of_returns.append(email_return('                                                                                                                            gnotter@jefferies.com '))
list_of_returns.append(email_return('fda              ronnie@gmail.com'))
list_of_returns.append(email_return(' AAPL | $263.19                                                      amit.daryanani@evercoreisi.com          irvin.liu@evercoreisi.com'))
print(list_of_returns)


[['Andrew.Uerkwitz@opco.com', 'Martin.Yang@opco.com'], ['Paul.Dean@opco.com'], ['James.Faucette@morganstanley.com'], ['gnotter@jefferies.com'], ['ronnie@gmail.com'], ['amit.daryanani@evercoreisi.com', 'irvin.liu@evercoreisi.com']]


In [206]:
#Output Raw Data to a csv File for more cleaning
def outputToCSV(ticker, data_dictionary):
    path = '/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}.csv'.format(ticker)
    data_total = pd.DataFrame(data) #creates dataFrame with cols being those from data (dict)
    data_total.to_csv(path)

outputToCSV('XOM', data)

In [ ]:
#general how to - one cycle for each of the 25 companys - data dictionary and pageRange_list persist to feed info into each 
#stage of methods, the entire process is linear (and thus, pretty simple to understand)